# Wrangle ML datasets
- Explore tabular data for supervised machine learning
- Join relational data for supervised machine learning

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, \
                            plot_confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
from glob import glob # Searching & grouping filenames
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests # To make a request to a web page
import tarfile # Makes it possible to read and write tar archives

# I. Wrangle Data

In [ ]:
# Download data
def download(url):
    filename = url.split('/')[-1]
    print(f'Downloading {url}')
    r = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(r.content)
    print(f'Downloaded {filename}')

download('https://lambdaschool-ds-instruction.s3.amazonaws.com/datasets%3Ainstacart_2017_05_01.tar.gz')

Downloaded datasets%3Ainstacart_2017_05_01.tar.gz


In [ ]:
# # Uncompress data
tarfile.open('datasets%3Ainstacart_2017_05_01.tar.gz').extractall()

In [ ]:
# Print the csv filenames
pattern = 'instacart_2017_05_01/*.csv'

for file in glob(pattern):
  print(file)

instacart_2017_05_01/departments.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/products.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/order_products__prior.csv


**Before you start,** load each of the above `.csv` files into its own DataFrame.

In [ ]:
order_products__prior = pd.read_csv('instacart_2017_05_01/order_products__prior.csv')
order_products__train = pd.read_csv('instacart_2017_05_01/order_products__train.csv')
orders = pd.read_csv('instacart_2017_05_01/orders.csv')
products = pd.read_csv('instacart_2017_05_01/products.csv')

## I.a. Warm-up Questions

What information is contained in the column `orders['eval_set']`?

In [ ]:
orders['eval_set'].value_counts()

prior    3214874
train     131209
test       75000
Name: eval_set, dtype: int64

In [ ]:
orders['eval_set'].describe()

count     3421083
unique          3
top         prior
freq      3214874
Name: eval_set, dtype: object

The first row of `orders['order_id']` is `2539329`. Where can we find the items that were included in that order?

In [ ]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [ ]:
order_products__prior[order_products__prior['order_id'] == 2539329]


,order_id,product_id,add_to_cart_order,reordered
24076664,2539329,196,1,0
24076665,2539329,14084,2,0
24076666,2539329,12427,3,0
24076667,2539329,26088,4,0
24076668,2539329,26405,5,0


In [ ]:
product_id_items = order_products__prior[order_products__prior['order_id'] == 2539329]['product_id'].to_list()


In [ ]:
product_id_items

[196, 14084, 12427, 26088, 26405]

In [ ]:
products.loc[products['product_id'].isin(product_id_items)]

,product_id,product_name,aisle_id,department_id
195,196,Soda,77,7
12426,12427,Original Beef Jerky,23,19
14083,14084,Organic Unsweetened Vanilla Almond Milk,91,16
26087,26088,Aged White Cheddar Popcorn,23,19
26404,26405,XL Pick-A-Size Paper Towel Rolls,54,17


The first row of `order_products__prior['product_id']` is `33120`. What is the name of that product?

In [ ]:
products.loc[products['product_id'] == 33120 ]

,product_id,product_name,aisle_id,department_id
33119,33120,Organic Egg Whites,86,16


## I.b. Define Our Machine Learning Problem

- We want predict whether or not a customer will purchase a specific item (or our choosing).
- The most commonly ordered product: `'Banana'` (`24852`).
- Our model is going to predict whether or not an order will include a `'Banana'`.

Lets adjust the Kaggle competition classification task from "What products will be ordered?" (multiclass, multilabel classification) to "Will one product be reordered?" (binary classification).

In [ ]:
order_products__prior['product_id'].value_counts().head()

24852    472565
13176    379450
21137    264683
21903    241921
47209    213584
Name: product_id, dtype: int64

In [ ]:
mask = (products['product_id'] == 24852)
products[mask]

,product_id,product_name,aisle_id,department_id
24851,24852,Banana,24,4


## I.c. Create Feature Matrix and Target Vector

Our **feature matrix** will be all the `'train'` rows from `orders`.

In [ ]:
mask = (orders['eval_set'] == 'train')

X = orders[mask].copy()

X.drop(columns='eval_set', inplace=True)

In [ ]:
X.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
10,1187899,1,11,4,8,14.0
25,1492625,2,15,1,11,30.0
49,2196797,5,5,0,11,6.0
74,525192,7,21,2,11,6.0
78,880375,8,4,1,14,10.0


In [ ]:
# X['eval_set'].nunique()

In [ ]:
# X.shape

Our **target vector** will be whether or not each order in `X` contains the item we've chosen above.

In [ ]:
order_products__train['is_banana'] = order_products__train['product_id'] == 24852

In [ ]:
order_products__train.head()

,order_id,product_id,add_to_cart_order,reordered,is_banana
0,1,49302,1,1,False
1,1,11109,2,1,False
2,1,10246,3,0,False
3,1,49683,4,0,False
4,1,43633,5,1,False


In [ ]:
banana_orders = order_products__train[order_products__train['is_banana']]['order_id']

In [ ]:
 X['order_id'].isin(banana_orders).astype(int)

10         0
25         1
49         0
74         0
78         0
          ..
3420838    0
3420862    0
3420924    0
3420933    1
3421082    1
Name: order_id, Length: 131209, dtype: int64

In [ ]:
X['includes_bananas'] = X['order_id'].isin(banana_orders).astype(int)

In [ ]:
X['includes_bananas']

10         0
25         1
49         0
74         0
78         0
          ..
3420838    0
3420862    0
3420924    0
3420933    1
3421082    1
Name: includes_bananas, Length: 131209, dtype: int64

In [ ]:
X.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,includes_bananas
10,1187899,1,11,4,8,14.0,0
25,1492625,2,15,1,11,30.0,1
49,2196797,5,5,0,11,6.0,0
74,525192,7,21,2,11,6.0,0
78,880375,8,4,1,14,10.0,0


# I.d Feature Engineering

## What features can we engineer? We want to predict, will these customers reorder bananas on their next order?

- Products per order
- Time of day
- Have they reordered bananas before? (Have ordered bananas >= 2 times)
- Other fruit they buy
- Size of orders (customers with smaller orders on average are less likely to be reordering any particular product on their next order)


- Frequency of banana orders:
    - % of orders
    - Time between banana orders: Every n days on average
    - Raw count: Total orders, how many times have you ordered bananas?

- Recency of banana orders
    - n days since you ordered banana

How many items in the order?

In [ ]:
n_items_per_order = order_products__train.groupby('order_id')['product_id'].count().rename('n_items')

In [ ]:
n_items_per_order

order_id
1           8
36          8
38          9
96          7
98         49
           ..
3421049     6
3421056     5
3421058     8
3421063     4
3421070     3
Name: n_items, Length: 131209, dtype: int64

In [ ]:
X.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,includes_bananas
10,1187899,1,11,4,8,14.0,0
25,1492625,2,15,1,11,30.0,1
49,2196797,5,5,0,11,6.0,0
74,525192,7,21,2,11,6.0,0
78,880375,8,4,1,14,10.0,0


In [ ]:
X = X.merge(n_items_per_order,
        how='left',
        left_on='order_id',
        right_index=True
        )

In [ ]:
X.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,includes_bananas,n_items
10,1187899,1,11,4,8,14.0,0,11
25,1492625,2,15,1,11,30.0,1,31
49,2196797,5,5,0,11,6.0,0,9
74,525192,7,21,2,11,6.0,0,9
78,880375,8,4,1,14,10.0,0,18


Did the user order banana in previous orders?

In [ ]:
order_products__prior['is_banana'] = (order_products__prior['product_id'] == 24852)

banana_orders_id_prior = order_products__prior[order_products__prior['is_banana']]['order_id']

In [ ]:
prior_orders = orders[orders['eval_set'] == 'prior'].copy()

In [ ]:
prior_orders['has_banana'] = prior_orders['order_id'].isin(banana_orders_id_prior)

In [ ]:
prior_banana_user_ids = prior_orders[prior_orders['has_banana']]['user_id'].unique()

In [ ]:
X['prior_banana_orders'] = X['user_id'].isin(prior_banana_user_ids).astype(int)

In [ ]:
X.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,includes_bananas,n_items,prior_banana_orders
10,1187899,1,11,4,8,14.0,0,11,0
25,1492625,2,15,1,11,30.0,1,31,1
49,2196797,5,5,0,11,6.0,0,9,0
74,525192,7,21,2,11,6.0,0,9,0
78,880375,8,4,1,14,10.0,0,18,0


# II. Split Data

In [ ]:
target = 'includes_bananas'
y = X[target]
X = X.drop(columns=['order_id', 'user_id', 'order_number', target])

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# III. Establish Baseline

In [ ]:
 y_train.value_counts(normalize=True)

0    0.856993
1    0.143007
Name: includes_bananas, dtype: float64

In [ ]:
print('Baseline accuracy:', y_train.value_counts(normalize=True).max())

Baseline accuracy: 0.8569931502281669


# IV. Build Model

In [ ]:
model_rf = RandomForestClassifier(random_state=42, n_jobs=-1)

model_rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

# V. Check Metrics

In [ ]:
model_rf.score(X_train, y_train)

0.9464498366153172

In [ ]:
model_rf.score(X_val, y_val)

0.839836902675101